In [2]:
# -*- coding: utf-8 -*-
"""Airline_Passenger_Satisfaction_Preprocessing.ipynb

Prétraitement des données - Satisfaction Passagers Aériens
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 📁 SECTION 1: CHARGEMENT DES DONNÉES
# ============================================================================

print("="*80)
print("✈️ PRÉTRAITEMENT - SATISFACTION PASSAGERS AÉRIENS")
print("="*80)

# Charger les données
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"✅ Données chargées avec succès!")
print(f"📊 Train: {train_df.shape[0]:,} lignes × {train_df.shape[1]} colonnes")
print(f"📊 Test: {test_df.shape[0]:,} lignes × {test_df.shape[1]} colonnes")

# ============================================================================
# 🎯 SECTION 2: SUPPRESSION DES COLONNES INUTILES
# ============================================================================

print("\n" + "="*80)
print("🎯 SECTION 2: SUPPRESSION DES COLONNES INUTILES")
print("="*80)

# Colonnes à supprimer basées sur l'EDA
cols_to_drop = ['id']  # ID unique, pas informatif

# Supprimer les colonnes
train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df = test_df.drop(columns=cols_to_drop, errors='ignore')

print(f"✅ Colonnes supprimées: {cols_to_drop}")
print(f"📊 Dimensions après suppression:")
print(f"   Train: {train_df.shape[0]:,} lignes × {train_df.shape[1]} colonnes")
print(f"   Test: {test_df.shape[0]:,} lignes × {test_df.shape[1]} colonnes")

# ============================================================================
# 🔧 SECTION 3: GESTION DES VALEURS MANQUANTES
# ============================================================================

print("\n" + "="*80)
print("🔧 SECTION 3: GESTION DES VALEURS MANQUANTES")
print("="*80)

# Identifier les colonnes avec valeurs manquantes
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

missing_cols_train = missing_train[missing_train > 0].index.tolist()
missing_cols_test = missing_test[missing_test > 0].index.tolist()

if missing_cols_train or missing_cols_test:
    print("⚠️ Colonnes avec valeurs manquantes:")

    for df_name, df_missing, missing_cols in [("Train", missing_train, missing_cols_train),
                                              ("Test", missing_test, missing_cols_test)]:
        if missing_cols:
            print(f"\n🔸 {df_name}:")
            for col in missing_cols:
                missing_pct = (df_missing[col] / len(train_df if df_name == "Train" else test_df)) * 100
                print(f"   - {col}: {df_missing[col]} valeurs ({missing_pct:.2f}%)")

    # Stratégie d'imputation basée sur l'EDA
    print("\n🎯 Stratégie d'imputation:")

    # 1. Colonnes catégorielles: Mode
    categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
    cat_missing = [col for col in categorical_cols if col in missing_cols_train or col in missing_cols_test]

    if cat_missing:
        print("   🔸 Catégorielles: Imputation par mode")
        for col in cat_missing:
            if col in train_df.columns:
                mode_val = train_df[col].mode()[0]
                train_df[col] = train_df[col].fillna(mode_val)
            if col in test_df.columns:
                mode_val = test_df[col].mode()[0] if not test_df[col].mode().empty else train_df[col].mode()[0]
                test_df[col] = test_df[col].fillna(mode_val)

    # 2. Colonnes numériques: Médiane (robuste aux outliers)
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
    num_missing = [col for col in numeric_cols if col in missing_cols_train or col in missing_cols_test]

    if num_missing:
        print("   🔸 Numériques: Imputation par médiane")
        for col in num_missing:
            if col in train_df.columns:
                median_val = train_df[col].median()
                train_df[col] = train_df[col].fillna(median_val)
            if col in test_df.columns:
                median_val = test_df[col].median() if not test_df[col].isnull().all() else train_df[col].median()
                test_df[col] = test_df[col].fillna(median_val)

    # Vérifier après imputation
    print("\n✅ Après imputation:")
    print(f"   Train - Valeurs manquantes: {train_df.isnull().sum().sum()}")
    print(f"   Test - Valeurs manquantes: {test_df.isnull().sum().sum()}")
else:
    print("✅ Aucune valeur manquante détectée!")

# ============================================================================
# 🏷️ SECTION 4: ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================================================

print("\n" + "="*80)
print("🏷️ SECTION 4: ENCODAGE DES VARIABLES CATÉGORIELLES")
print("="*80)

# Identifier les colonnes catégorielles
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
target_col = 'satisfaction'

# Retirer la cible des variables catégorielles
if target_col in categorical_cols:
    categorical_cols.remove(target_col)

print(f"🔍 Variables catégorielles à encoder: {categorical_cols}")

# Dictionnaire pour stocker les encodeurs
encoders = {}

# Encodage spécifique basé sur l'analyse de l'EDA
for col in categorical_cols:
    print(f"\n🔸 Encodage de '{col}':")

    # Encodage binaire pour les variables à 2 catégories
    if train_df[col].nunique() == 2:
        print(f"   ⚡ Encodage binaire (Label Encoding)")
        le = LabelEncoder()

        # Appliquer sur train
        train_df[col] = le.fit_transform(train_df[col])

        # Appliquer sur test en utilisant les mêmes classes
        # Pour les nouvelles catégories dans test, on utilise une valeur par défaut
        test_categories = set(test_df[col].unique())
        train_categories = set(le.classes_)

        if not test_categories.issubset(train_categories):
            print(f"   ⚠️  Catégories inconnues dans test: {test_categories - train_categories}")
            # Remplacer les catégories inconnues par la plus fréquente
            most_frequent = train_df[col].mode()[0]
            test_df[col] = test_df[col].apply(lambda x: x if x in le.classes_ else le.classes_[most_frequent])

        test_df[col] = le.transform(test_df[col])
        encoders[col] = le

        print(f"   📊 Mapping: {dict(zip(le.classes_, range(len(le.classes_))))}")

    # One-Hot Encoding pour les variables à >2 catégories
    else:
        print(f"   🎯 One-Hot Encoding ({train_df[col].nunique()} catégories)")

        # Créer les variables dummy
        dummies_train = pd.get_dummies(train_df[col], prefix=col, drop_first=True)
        dummies_test = pd.get_dummies(test_df[col], prefix=col, drop_first=True)

        # S'assurer que test a les mêmes colonnes que train
        missing_cols = set(dummies_train.columns) - set(dummies_test.columns)
        for c in missing_cols:
            dummies_test[c] = 0

        # Garder seulement les colonnes dans le même ordre
        dummies_test = dummies_test[dummies_train.columns]

        # Supprimer la colonne originale et ajouter les dummies
        train_df = pd.concat([train_df.drop(columns=[col]), dummies_train], axis=1)
        test_df = pd.concat([test_df.drop(columns=[col]), dummies_test], axis=1)

        print(f"   📊 Colonnes créées: {list(dummies_train.columns)}")

print(f"\n✅ Encodage terminé!")
print(f"📊 Nouvelles dimensions:")
print(f"   Train: {train_df.shape[0]:,} lignes × {train_df.shape[1]} colonnes")
print(f"   Test: {test_df.shape[0]:,} lignes × {test_df.shape[1]} colonnes")

# ============================================================================
# 🎯 SECTION 5: ENCODAGE DE LA VARIABLE CIBLE
# ============================================================================

print("\n" + "="*80)
print("🎯 SECTION 5: ENCODAGE DE LA VARIABLE CIBLE")
print("="*80)

# Encoder la variable cible
print("🔸 Encodage de la variable cible 'satisfaction'")
target_encoder = LabelEncoder()
train_df[target_col] = target_encoder.fit_transform(train_df[target_col])

# Vérifier si la variable cible existe dans test (pour l'évaluation)
if target_col in test_df.columns:
    test_df[target_col] = target_encoder.transform(test_df[target_col])

print(f"📊 Mapping cible: {dict(zip(target_encoder.classes_, target_encoder.transform(target_encoder.classes_)))}")
print(f"   - 0: {target_encoder.classes_[0]} (classe majoritaire)")
print(f"   - 1: {target_encoder.classes_[1]} (classe minoritaire)")

# ============================================================================
# 📈 SECTION 6: GESTION DES OUTLIERS
# ============================================================================

print("\n" + "="*80)
print("📈 SECTION 6: GESTION DES OUTLIERS")
print("="*80)

# Identifier les variables numériques (après encodage)
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# Variables avec forte asymétrie détectées dans l'EDA
skewed_cols = ['Departure Delay in Minutes', 'Arrival Delay in Minutes']

print("🔍 Gestion des outliers via winsorization ou transformation:")

for col in numeric_cols:
    if col in skewed_cols:
        print(f"🔸 {col}: Forte asymétrie détectée → Transformation log")

        # Ajouter 1 pour éviter log(0)
        train_df[f'{col}_log'] = np.log1p(train_df[col])
        test_df[f'{col}_log'] = np.log1p(test_df[col])

        # Supprimer la colonne originale
        train_df = train_df.drop(columns=[col])
        test_df = test_df.drop(columns=[col])
    else:
        # Winsorization pour les autres variables avec outliers
        Q1 = train_df[col].quantile(0.01)
        Q3 = train_df[col].quantile(0.99)

        outliers_count = ((train_df[col] < Q1) | (train_df[col] > Q3)).sum()
        if outliers_count > 0 and outliers_count < len(train_df) * 0.05:
            print(f"🔸 {col}: {outliers_count} outliers → Winsorization (1%-99%)")

            # Appliquer winsorization
            train_df[col] = train_df[col].clip(Q1, Q3)
            test_df[col] = test_df[col].clip(Q1, Q3)

print("\n✅ Gestion des outliers terminée!")

# ============================================================================
# ⚖️ SECTION 7: NORMALISATION/STANDARDISATION
# ============================================================================

print("\n" + "="*80)
print("⚖️ SECTION 7: NORMALISATION/STANDARDISATION")
print("="*80)

# Identifier les colonnes à normaliser
features_to_scale = [col for col in train_df.columns if col != target_col]

print(f"🔍 {len(features_to_scale)} features à normaliser")

# Séparer features et target
X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

X_test = test_df.drop(columns=[target_col], errors='ignore')
y_test = test_df[target_col] if target_col in test_df.columns else None

# Appliquer RobustScaler (robuste aux outliers)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir en DataFrame pour conserver les noms de colonnes
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("✅ Normalisation avec RobustScaler terminée!")

# ============================================================================
# 🔍 SECTION 8: SÉLECTION DE FEATURES
# ============================================================================

print("\n" + "="*80)
print("🔍 SECTION 8: SÉLECTION DE FEATURES")
print("="*80)

# 1. Supprimer les features avec variance quasi-nulle
print("1️⃣  Suppression des features à variance nulle:")
selector_variance = VarianceThreshold(threshold=0.01)
X_train_var = selector_variance.fit_transform(X_train_scaled_df)
X_test_var = selector_variance.transform(X_test_scaled_df)

# Récupérer les noms des colonnes conservées
selected_features = X_train_scaled_df.columns[selector_variance.get_support()]
print(f"   ✅ {len(selected_features)} features conservées sur {X_train_scaled_df.shape[1]}")

# 2. Sélection basée sur les tests statistiques (ANOVA F-value)
print("\n2️⃣  Sélection basée sur ANOVA F-value:")
k = min(20, len(selected_features))  # Garder les 20 meilleures features
selector_kbest = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector_kbest.fit_transform(X_train_scaled_df[selected_features], y_train)
X_test_selected = selector_kbest.transform(X_test_scaled_df[selected_features])

# Récupérer les noms des colonnes sélectionnées
selected_features_kbest = selected_features[selector_kbest.get_support()]

print(f"   ✅ Top {k} features sélectionnées:")
scores = selector_kbest.scores_
indices = np.argsort(scores)[::-1]

for i in range(min(10, len(scores))):
    idx = indices[i]
    print(f"      {i+1:2d}. {selected_features[idx]:30} : {scores[idx]:.2f}")

# Mettre à jour les DataFrames
X_train_final = pd.DataFrame(X_train_selected, columns=selected_features_kbest, index=X_train.index)
X_test_final = pd.DataFrame(X_test_selected, columns=selected_features_kbest, index=X_test.index)

print(f"\n📊 Dimensions finales:")
print(f"   X_train: {X_train_final.shape}")
print(f"   X_test: {X_test_final.shape}")

# ============================================================================
# 💾 SECTION 9: SAUVEGARDE DES DONNÉES PRÉTRAITÉES
# ============================================================================

print("\n" + "="*80)
print("💾 SECTION 9: SAUVEGARDE DES DONNÉES PRÉTRAITÉES")
print("="*80)

# Sauvegarder les données prétraitées
output_path = 'processed/'

# Créer le dossier si nécessaire
import os
os.makedirs(output_path, exist_ok=True)

# Sauvegarder les données
X_train_final.to_csv(f'{output_path}X_train_processed.csv', index=False)
X_test_final.to_csv(f'{output_path}X_test_processed.csv', index=False)

y_train.to_csv(f'{output_path}y_train_processed.csv', index=False)
if y_test is not None:
    y_test.to_csv(f'{output_path}y_test_processed.csv', index=False)

# Sauvegarder les objets de preprocessing
import joblib
preprocessing_objects = {
    'scaler': scaler,
    'variance_selector': selector_variance,
    'kbest_selector': selector_kbest,
    'target_encoder': target_encoder,
    'encoders': encoders,
    'selected_features': selected_features_kbest.tolist()
}

joblib.dump(preprocessing_objects, f'{output_path}preprocessing_objects.pkl')

print("✅ Données sauvegardées avec succès!")
print(f"📁 Dossier: {output_path}")
print(f"📄 Fichiers générés:")
print(f"   - X_train_processed.csv")
print(f"   - X_test_processed.csv")
print(f"   - y_train_processed.csv")
print(f"   - y_test_processed.csv (si disponible)")
print(f"   - preprocessing_objects.pkl")

# ============================================================================
# 📊 SECTION 10: RÉSUMÉ DU PRÉTRAITEMENT
# ============================================================================

print("\n" + "="*80)
print("📊 SECTION 10: RÉSUMÉ DU PRÉTRAITEMENT")
print("="*80)

print("🎯 ÉTAPES EFFECTUÉES:")
print("""
1. ✅ Chargement des données (train.csv, test.csv)
2. ✅ Suppression des colonnes inutiles (id)
3. ✅ Gestion des valeurs manquantes (imputation médiane/mode)
4. ✅ Encodage des variables catégorielles:
   - Label Encoding pour variables binaires
   - One-Hot Encoding pour variables multi-classes
5. ✅ Encodage de la variable cible (satisfaction → 0/1)
6. ✅ Gestion des outliers:
   - Transformation log pour variables très asymétriques
   - Winsorization pour outliers modérés
7. ✅ Normalisation avec RobustScaler
8. ✅ Sélection de features:
   - VarianceThreshold (suppression features à variance quasi-nulle)
   - SelectKBest (sélection des 20 meilleures features)
9. ✅ Sauvegarde des données prétraitées
""")

print("📈 STATISTIQUES FINALES:")
print(f"   - Nombre de features initial: {train_df.shape[1] - 1}")
print(f"   - Nombre de features final: {X_train_final.shape[1]}")
print(f"   - Réduction: {((train_df.shape[1] - 1 - X_train_final.shape[1]) / (train_df.shape[1] - 1) * 100):.1f}%")
print(f"   - Taille échantillon train: {X_train_final.shape[0]:,}")
print(f"   - Taille échantillon test: {X_test_final.shape[0]:,}")

# Aperçu des données prétraitées
print("\n🔍 APERÇU DES DONNÉES PRÉTRAITÉES (X_train):")
print(X_train_final.head())
print(f"\n🔍 APERÇU DE LA CIBLE (y_train):")
print(y_train.head())

print("\n" + "="*80)
print("✅ PRÉTRAITEMENT TERMINÉ - PRÊT POUR LA MODÉLISATION!")
print("="*80)

✈️ PRÉTRAITEMENT - SATISFACTION PASSAGERS AÉRIENS
✅ Données chargées avec succès!
📊 Train: 103,904 lignes × 24 colonnes
📊 Test: 25,976 lignes × 24 colonnes

🎯 SECTION 2: SUPPRESSION DES COLONNES INUTILES
✅ Colonnes supprimées: ['id']
📊 Dimensions après suppression:
   Train: 103,904 lignes × 23 colonnes
   Test: 25,976 lignes × 23 colonnes

🔧 SECTION 3: GESTION DES VALEURS MANQUANTES
⚠️ Colonnes avec valeurs manquantes:

🔸 Train:
   - Arrival Delay in Minutes: 310 valeurs (0.30%)

🔸 Test:
   - Arrival Delay in Minutes: 83 valeurs (0.32%)

🎯 Stratégie d'imputation:
   🔸 Numériques: Imputation par médiane

✅ Après imputation:
   Train - Valeurs manquantes: 0
   Test - Valeurs manquantes: 0

🏷️ SECTION 4: ENCODAGE DES VARIABLES CATÉGORIELLES
🔍 Variables catégorielles à encoder: ['Gender', 'Customer Type', 'Type of Travel', 'Class']

🔸 Encodage de 'Gender':
   ⚡ Encodage binaire (Label Encoding)
   📊 Mapping: {'Female': 0, 'Male': 1}

🔸 Encodage de 'Customer Type':
   ⚡ Encodage binaire (L